In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import os
import datetime
import sys

from models import model_classifier
from utils_dir.utils import *
import config

if config.ESC_10:
    import dataset_ESC10 as dataset
elif config.ESC_50:
    import dataset_ESC50 as dataset
elif config.US8K:
    import dataset_US8K as dataset

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Update the model loading line
model = torchvision.models.resnet50(pretrained=True).to(device)
model.fc = nn.Sequential(nn.Identity())

model = nn.DataParallel(model, device_ids=[0])
model = model.to(device)

classifier = model_classifier.Classifier().to(device)

train_loader, val_loader = dataset.create_generators()

root = './results/'
main_path = root + str(datetime.datetime.now().strftime('%Y-%m-%d')) + "_crossEntropyLoss"
if not os.path.exists(main_path):
    os.mkdir(main_path)

classifier_path = main_path + '/' + 'classifier'

# Modify the code that creates directories to handle existing directories
if not os.path.exists(classifier_path):
    os.mkdir(classifier_path)
else:
    print(f"Directory {classifier_path} already exists.")

optimizer = torch.optim.AdamW(list(model.parameters()) + list(classifier.parameters()),
                             lr=config.lr, weight_decay=1e-3)

scheduler = WarmUpExponentialLR(optimizer, cold_epochs=0, warm_epochs=config.warm_epochs, gamma=config.gamma)


def hotEncoder(v):
    ret_vec = torch.zeros(v.shape[0], config.class_numbers).to(device)
    for s in range(v.shape[0]):
        ret_vec[s][v[s]] = 1
    return ret_vec

def cross_entropy_one_hot(input, target):
    _, labels = target.max(dim=1)
    return nn.CrossEntropyLoss(weight=class_weights)(input, labels)

c:\Users\Gabriel\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Gabriel\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Directory ./results/2023-09-21_crossEntropyLoss/classifier already exists.


In [4]:
torch.cuda.empty_cache()
print(config.ESC_50)
print(config.ESC_10)
print(config.US8K)

False
False
True


In [ ]:


###########################################################################################
#create class weight vector with the length of number of classes with 0
class_weights = torch.ones(config.class_numbers).to(device)

main_model = model.module if hasattr(model, 'module') else model
###########################################################################################

def train_crossEntropy():
	num_epochs = 800
	with open(main_path + '/results.txt','w', 1) as output_file:
		mainModel_stopping = EarlyStopping(patience=300, verbose=True, log_path=main_path, output_file=output_file)
		classifier_stopping = EarlyStopping(patience=300, verbose=False, log_path=classifier_path, output_file=output_file)

		print('*****', file=output_file)
		print('BASELINE', file=output_file)
		print('transfer - augmentation on both waves and specs - 3 channels', file=output_file)
		if config.ESC_10:
			print('ESC_10', file=output_file)
			print('train folds are {} and test fold is {}'.format(config.train_folds, config.test_fold), file=output_file)
		elif config.ESC_50:
			print('ESC_50', file=output_file)
			print('train folds are {} and test fold is {}'.format(config.train_folds, config.test_fold), file=output_file)
		elif config.US8K:
			print('US8K', file=output_file)
			print('train folds are {} and test fold is {}'.format(config.us8k_train_folds, config.us8k_test_fold), file=output_file)


		print('number of freq masks are {} and their max length is {}'.format(config.freq_masks, config.freq_masks_width), file=output_file)
		print('number of time masks are {} and their max length is {}'.format(config.time_masks, config.time_masks_width), file=output_file)
		print('*****', file=output_file)
	


		for epoch in range(num_epochs):
			model.train()
			classifier.train()
        
			train_loss = []
			train_corrects = 0
			train_samples_count = 0
        
			for _, x, label in train_loader:
				loss = 0
				optimizer.zero_grad()
            
				inp = x.float().to(device)
				label = label.to(device).unsqueeze(1)
				label_vec = hotEncoder(label)
            
				y_rep = model(inp)
				y_rep = F.normalize(y_rep, dim=0)
            
				y_pred = classifier(y_rep)
            
				loss += cross_entropy_one_hot(y_pred, label_vec)
				loss.backward()
				train_loss.append(loss.item() )
				optimizer.step()
            
				train_corrects += (torch.argmax(y_pred, dim=1) == torch.argmax(label_vec, dim=1)).sum().item()
				train_samples_count += x.shape[0]
        
        
			val_loss = []
			val_corrects = 0
			val_samples_count = 0
        
			model.eval()
			classifier.eval()
        
			with torch.no_grad():
				for _, val_x, val_label in val_loader:
					inp = val_x.float().to(device)
					label = val_label.to(device)
					label_vec = hotEncoder(label)
                
					y_rep = model(inp)
					y_rep = F.normalize(y_rep, dim=0)

					y_pred = classifier(y_rep)
                
					temp = cross_entropy_one_hot(y_pred, label_vec)
					val_loss.append(temp.item() )
                
					val_corrects += (torch.argmax(y_pred, dim=1) == torch.argmax(label_vec, dim=1)).sum().item() 
					val_samples_count += val_x.shape[0]
        
		
        
			scheduler.step()
        
			train_acc = train_corrects / train_samples_count
			val_acc = val_corrects / val_samples_count
			print('\n', file=output_file)
			print("Epoch: {}/{}...".format(epoch+1, num_epochs), "Loss: {:.4f}...".format(np.mean(train_loss)),
				"Val Loss: {:.4f}".format(np.mean(val_loss)), file=output_file)
			print('train_acc is {:.4f} and val_acc is {:.4f}'.format(train_acc, val_acc), file=output_file)
			mainModel_stopping(-val_acc, main_model, epoch+1)
			classifier_stopping(-val_acc, classifier, epoch+1)
			if mainModel_stopping.early_stop:
				print("Early stopping", file=output_file)
				return


if __name__ == "__main__":
	train_crossEntropy()

In [3]:
# Correct the instantiation of the classifier object
#classifier = Classifier().to(device) # Ensure Classifier class is defined before this line

# Get the main model from the DataParallel module
main_model = model.module if hasattr(model, 'module') else model

# Load the checkpoints
main_model.load_state_dict(torch.load('results/2023-09_crossEntropyLoss10_f/checkpoint.pt'))
classifier.load_state_dict(torch.load('results/2023-09_crossEntropyLoss10_f/classifier/checkpoint.pt'))


<All keys matched successfully>

In [4]:

from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
# Initialize variables to store the true and predicted labels
true_labels = []
pred_labels = []

# Evaluate the model on the test dataset
with torch.no_grad():
    for _, x, label in val_loader:
        inp = x.float().to(device)
        label = label.to(device).unsqueeze(1)
        label_vec = hotEncoder(label)
        
        y_rep = main_model(inp)
        y_rep = F.normalize(y_rep, dim=0)

        y_pred = classifier(y_rep)
        
        true_labels.extend(label.squeeze(1).cpu().numpy())
        pred_labels.extend(torch.argmax(y_pred, dim=1).cpu().numpy())

# Calculate accuracy
correct_preds = sum(t == p for t, p in zip(true_labels, pred_labels))
accuracy = correct_preds / len(true_labels)

# Calculate precision, recall, F1-score, and support
precision, recall, f1_score, support = precision_recall_fscore_support(true_labels, pred_labels, average='weighted')

# Calculate the confusion matrix
conf_mat = confusion_matrix(true_labels, pred_labels)

# Print the classification report
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-score: {f1_score * 100:.2f}%")

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_mat)




Accuracy: 87.50%
Precision: 89.94%
Recall: 87.50%
F1-score: 87.67%
Confusion Matrix:
[[6 0 0 0 0 0 2 0 0 0]
 [0 8 0 0 0 0 0 0 0 0]
 [0 0 6 2 0 0 0 0 0 0]
 [0 0 0 8 0 0 0 0 0 0]
 [0 0 0 0 8 0 0 0 0 0]
 [0 0 0 0 0 8 0 0 0 0]
 [0 0 0 0 0 1 7 0 0 0]
 [0 0 1 0 0 1 0 6 0 0]
 [0 0 1 0 0 0 0 0 7 0]
 [0 0 0 2 0 0 0 0 0 6]]


In [10]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Ensure you have your config file imported
import config

# Assuming your dataset script is named dataset.py
#import dataset_US8K as downstream_dataset

# Define a simple CNN model (replace this with your preferred model)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 63 * 127, 10)  # We will update this later

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 63 * 127)  # We will update this later
        x = self.fc1(x)
        return x

    def get_conv_output_shape(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        return x.shape

# Initialize the model, define the loss function and the optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Create data generators
train_loader, test_loader = dataset.create_generators()

# Get the shape of the output after the conv and pool layers
dataiter = iter(train_loader)
images, labels, _ = next(dataiter)
output_shape = model.get_conv_output_shape(images[0].to(device))
print(output_shape)  # Print the output shape

# Now update the input dimensions of your Linear layer and view function
num_flat_features = output_shape[1] * output_shape[2] * output_shape[3]
model.fc1 = nn.Linear(num_flat_features, 10)  # Update the Linear layer with the correct input dimensions

def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = x.view(-1, num_flat_features)  # Update the view function with the correct dimensions
    x = self.fc1(x)
    return x
# Train the model
def train_model(num_epochs):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[1].to(device), data[2].to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 100 == 99:
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100}")
                running_loss = 0.0

    print('Finished Training')

# Run the training function for a defined number of epochs
train_model(num_epochs=5)

# Here after training, you would save your model and/or evaluate it on your test dataset


AttributeError: 'str' object has no attribute 'to'